reinforcement-learning/DP/Policy Evaluation.ipynb
Start jupyter from inside ~/github/reinforcement-learning 
greeness-macbookpro:reinforcement-learning greeness$ jupyter notebook
so that we can import lib.envs.gridworld as below.

Copied from gridworld.py
"""
Grid World environment from Sutton's Reinforcement Learning book chapter 4.
You are an agent on an MxN grid and your goal is to reach the terminal
state at the top left or the bottom right corner.
For example, a 4x4 grid looks as follows:
T  o  o  o
o  x  o  o
o  o  o  o
o  o  o  T
x is your position and T are the two terminal states.
You can take actions in each direction (UP=0, RIGHT=1, DOWN=2, LEFT=3).
Actions going off the edge leave you in your current state.
You receive a reward of -1 at each step until you reach a terminal state.
"""

In [1]:
import numpy as np
from lib.envs.gridworld import GridworldEnv
env = GridworldEnv()

In [2]:
print env.observation_space

Discrete(16)


In [3]:
print env.action_space

Discrete(4)


In [4]:
print dir(env)

['P', '__class__', '__del__', '__delattr__', '__dict__', '__doc__', '__format__', '__getattribute__', '__hash__', '__init__', '__module__', '__new__', '__reduce__', '__reduce_ex__', '__repr__', '__setattr__', '__sizeof__', '__str__', '__subclasshook__', '__weakref__', '_close', '_closed', '_env_closer_id', '_owns_render', '_render', '_reset', '_seed', '_step', 'action_space', 'close', 'configure', 'isd', 'lastaction', 'metadata', 'monitor', 'nA', 'nS', 'np_random', 'observation_space', 'render', 'reset', 'reward_range', 's', 'seed', 'shape', 'step', 'unwrapped']


In [5]:
print env.reward_range

(-inf, inf)


In [6]:
print env.nA, env.nS, env.s, env.lastaction
print env.metadata

4 16 11 None
{'render.modes': ['human', 'ansi']}


In [7]:
# state 0 and state 15 are the two terminal states
# Once you are at the two states (P[0] or P[15]), you transfer to the same state with probability of 1.0 and
# reward 0 and done = true no mater what action you take (0, 1, 2, or 3).
print env.P[0]
print env.P[15]

{0: [(1.0, 0, 0.0, True)], 1: [(1.0, 0, 0.0, True)], 2: [(1.0, 0, 0.0, True)], 3: [(1.0, 0, 0.0, True)]}
{0: [(1.0, 15, 0.0, True)], 1: [(1.0, 15, 0.0, True)], 2: [(1.0, 15, 0.0, True)], 3: [(1.0, 15, 0.0, True)]}


In [8]:
# in other states, e.g. state 12 (fourth row, first column)
# Actions going off the edge (action 2-DOWN or 3-LEFT) leave you in your current state 
# Actions in the valid direction moves you to the neighboring cell. (12->8 by action 0-UP and 12->13 by action 1-RIGHT)
print env.P[12]

{0: [(1.0, 8, -1.0, False)], 1: [(1.0, 13, -1.0, False)], 2: [(1.0, 12, -1.0, False)], 3: [(1.0, 12, -1.0, False)]}


In [9]:
def policy_eval(policy, env, discount_factor=1.0, theta=0.00001):
    """
    Evaluate a policy given an environment and a full description of the environment's dynamics.
    
    Args:
        policy: [S, A] shaped matrix representing the policy.
        env: OpenAI env. env.P represents the transition probabilities of the environment.
            env.P[s][a] is a (prob, next_state, reward, done) tuple.
        theta: We stop evaluation once our value function change is less than theta for all states.
        discount_factor: gamma discount factor.
    
    Returns:
        Vector of length env.nS representing the value function.
    """
    # Start with a random (all 0) value function
    V = np.zeros(env.nS)
    nIterations = 0
    while True:
        # TODO: Implement!
        Delta = 0
        for s in range(env.nS):
            oldv = V[s]
            newv = 0
            for a, action_prob in enumerate(policy[s]):
                for prob, next_state, reward, done in env.P[s][a]:
                    newv += action_prob * prob * (reward + discount_factor * V[next_state])
            Delta = max(Delta, abs(oldv - newv))
            V[s] = newv
        nIterations += 1
        if nIterations % 10 == 0: 
            print Delta
        if Delta < theta:                
          break
    
    print nIterations
    return np.array(V)

In [10]:
print env.P[1][1]

[(1.0, 2, -1.0, False)]


In [11]:
random_policy = np.ones([env.nS, env.nA]) / env.nA
v = policy_eval(random_policy, env)

0.8924059503
0.372594031843
0.155274167545
0.0647083950673
0.0269663422805
0.0112378558487
0.00468322335909
0.00195166954678
0.000813331700794
0.000338944908272
0.000141250673906
5.88642944344e-05
2.45308929401e-05
1.02229155097e-05
141


In [12]:
expected_v = np.array([0, -14, -20, -22, -14, -18, -20, -20, -20, -20, -18, -14, -22, -20, -14, 0])
np.testing.assert_array_almost_equal(v, expected_v, decimal=2)

In [13]:
print v

[  0.         -13.99993529 -19.99990698 -21.99989761 -13.99993529
 -17.9999206  -19.99991379 -19.99991477 -19.99990698 -19.99991379
 -17.99992725 -13.99994569 -21.99989761 -19.99991477 -13.99994569   0.        ]
